# Reconstruction of Reflectance Spectra
### An example of dimensionality reduction whilst preserving key characteristics

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from eigen_functions import power_method, deflate

tol = 1e-6

#### Loading Raw Data
Read in CSV file of reflectance spectra for both types of beans recorded using a Fourier-Transform Infrared Spectrometer (FTIR). The first row in the dataset contains the wavelengths at which the reflectances are measured.

In [27]:
# Read in spectra data
dataset = pd.read_csv("FTIR_Spectra_Coffee.csv", header = 0)
spectra = np.array(dataset)
wavelengths = list(dataset.columns.values)

# Get shape of dataset
rows, cols = np.shape(spectra)[0], np.shape(spectra)[1]

#### Find Covariance Matrix
Calculate the covariance matrix from the reflectance spectra using pre-defined functions.

In [ ]:
# Center the spectra

# Find covariance matrix


#### Find Eigenpairs of Spectra Matrix
Calculate Eigenvalues and Eigenvectors using pre-defined functions.

In [ ]:
# Create initial vector to aid with matrix calculations
init_vector = np.ones(cols)

eigvals = []
eigvecs = []
# Loop through the spectra and find Eigenpairs and store as arrays
for i in range(rows):
    tempval, tempvec = power_method(cov, init_vector, tol)
    cov = deflate(cov, tempval, tempvec)
    eigvals.append(tempval)
    eigvecs.append(tempvec)

#### Examine explained variation using Eigenpairs
Find how much of the original reflectance spectra is captured by its Eigenpairs.

In [ ]:
# Find total sum of Eigenvalues
eigsum = sum(eigvals)[0]
eigval = []
for i in range(0, len(eigvals)):
    eigval.append(eigvals[i][0])

# Find cumulative sum of Eigenvalues
eigratio = np.cumsum(eigval)/eigsum
x = np.array(range(1, len(eigvals) + 1))

In [ ]:
# Plot explained variance
plt.plot(x, eigratio, "b+")
plt.plot(x, [0.995]*len(x), "r-")   # Plot horizontal line for comparison
plt.ylabel("Explained Variance Ratio")
plt.xlabel("Number of Eigenvectors")
plt.title("Explained Variance Ratio against the Number of Eigenvectors")
plt.show()

In [ ]:
# Calculate gradient between consecutive points to find the limiting Eigenvalue


Based on the results above, we can use **X** Eigenvalues to explain around **99.Y%** of the variation in the original spectra.

In [ ]:
# Extract X principal components
prin = eigvecs[0:6, 0:286]

#### Carry out PCA
Apply PCA to generate a projected matrix by decomposed the original spectra into a cross-correlation matrix and compute the principal 
components. Project this spectra into Eigenspace and find a distinct pair of Eigenvectors to distinguish between the coffee beans.

In [ ]:
# Create centered matrix based on size of original spectra
rows, cols = np.shape(spectra)[0], np.shape(spectra)[1]
emp_mean = []
centered = np.zeros((rows, cols))

# Find the centered spectra matrix
for i in range(0, rows):
    emp_mean.append(np.mean(spectra[1:rows + 1, i]))
    for j in range(0, rows - 1):
        centered[j][i] = spectra[j + 1][i] - emp_mean[i]

In [ ]:
# Carry out matrix multiplication for projection
proj = np.matmul(centered, prin.transpose())

In [ ]:
# Obtain all combinations of columns in the projection matrix 
fig = plt.figure()
k = 1
for i in range(0, 5):
    for j in range(i + 1, 6):
        ax = fig.add_subplot(3, 5, k)
        ax.plot(proj[0:29, j], proj[0:29, i], "r.", label = "Arabica Coffee")
        ax.plot(proj[29:rows, j], proj[29:rows, i], "b.", label = "Robusta Coffee")
        ax.set_ylabel("PC" + str(i + 1), size = 10)
        ax.set_xlabel("PC" + str(j + 1), size = 10)
        k += 1

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc = "upper left", prop = {"size": 9})
fig.suptitle("Comparison of principal Components")
fig.tight_layout(pad = 1.5)
plt.show()

Third and fourth components (scatter plot 10) offer a good separation of coffee beans.

In [ ]:
# Scatter plot 10
plt.plot(proj[0:mid_rows, 3], proj[0:mid_rows, 2], "r.", label = "Arabica Coffee")
plt.plot(proj[mid_rows:rows, 3], proj[mid_rows:rows, 2], "b.", label = "Robusta Coffee")
plt.ylabel("PC3")
plt.xlabel("PC4")
plt.title("Comparison of principal Components 3 and 4")
plt.legend()
plt.show()

Select row 10 (Arabica) and row 39 (Robusta) for reconstruction.

In [ ]:
fig, axs = plt.subplots(2)

#### Generate Reconstructed Spectra
Based on the projected matrix, reconstruct the original spectra using X principal components.

In [ ]:
# Arabica
sums = [0]*cols
for i in range(0, 6):
    sums += proj[9, i]*eigvecs[i, 0:cols]

recon_arab = sums + emp_mean
axs[0].plot(spectra[0, :], recon_arab, label = "Reconstruction")
axs[0].plot(spectra[0, :], spectra[10, :], label = "Original")
axs[0].set_ylabel("Reflectance for Arabica")

# Robusta
sums = [0]*cols
for i in range(0, 6):
    sums += proj[38, i]*eigvecs[i, 0:cols]

recon_rob = sums + emp_mean
axs[1].plot(spectra[0, :], recon_rob, label = "Reconstruction")
axs[1].plot(spectra[0, :], spectra[39, :], label = "Original")
axs[1].set_ylabel("Reflectance for Robusta")
axs[1].set_xlabel("Wavelength (nm)")
fig.suptitle("Original and Reconstructed Spectra for Arabica and Robusta Coffees")
plt.legend()
plt.show()